# Домашнее задание 3 по теме: определение темы через Gensim и TF_IDF

Импортируем необходимые модули.

In [59]:
import pandas as pd
import numpy as np
import re
import gensim
from gensim.utils import simple_preprocess
import spacy
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

## Предобработка

Скачиваем датасет и преобразуем его в DataFrame. Дальнейший код представляет описывает препроцессинг перед обучением LDA Mallet модели, он, по сути, взят с семинара по Gensim.

In [60]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')

Очищаем тексты.

In [61]:
data = df.content.values.tolist()
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

Токенизация текстов и подготовка модели биграмм.

In [62]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data_words = list(sent_to_words(data))
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

Очистка от стоп-слов. Создание биграмм и лемматизация.

In [63]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [64]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [65]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

## Топикализация

Подготовка списков лемм к вводу в модель.

In [66]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

Функция нахождения наиболее оптимального числа топиков с помощью *CoherenceModel*. Используемая модель топикализации – *LDAMallet*.

In [73]:
def best_num():
    results = []
    for num_topics in tqdm(range(1, 50, 5)):
        mallet_path = '/Users/romankazakov/Downloads/mallet-2.0.8/bin/mallet'
        ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
        results.append(tuple([num_topics, coherence_model_ldamallet.get_coherence()]))
    best = (0, 0)
    for a in results:
        if a[1] > best[1]:
            best = a
    return best

In [75]:
res = best_num()
print('Таким образом, наиболее оптимальное число топиков:', res[0], 
      '(coherence score равен', str(res[1]) + ').')

100%|██████████| 10/10 [16:37<00:00, 99.72s/it] 

Таким образом, наиболее оптимальное число топиков: 21 (coherence score равен 0.5375255460843703).


Итак, **оптимальное число топиков — 21**. Теперь создадим ещё одну модель *LDAMallet*, чтобы работать именно с ней.

In [67]:
mallet_path = '/Users/romankazakov/Downloads/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=21, id2word=id2word)

Удаляем лишние столбцы из DataFrame.

In [68]:
del df['target']
del df['target_names']

Получаем список всех топиков и по 10 самых "важных" слов для каждого из них.

In [69]:
all_topics = ldamallet.show_topics(num_topics =-1, formatted=False)

С помощью счётчика весов слов, соответствующих определённым топикам, определяем широкие топики для каждого текста и записываем их в DataFrame вместе со списками лемм.

In [70]:
counter = 0
for text in texts:
    dic = {}
    for word in text:
        for topic in all_topics:
            for key_word in topic[1]:
                if key_word[0] == word:
                    if topic[0] in dic:
                        dic[topic[0]] += key_word[1]
                    else:
                        dic[topic[0]] = key_word[1]
    if dic != {}:
        score = max(list(dic.values()))
        for k, v in list(dic.items()):
            if v == score:
                main_topic = k
                break
    else:
        main_topic = np.nan
    df.loc[counter, 'topic'] = main_topic
    df.loc[counter, 'lemmas'] = [[t] for t in texts[counter]]
    counter += 1

## TF_IDF

Определение TF_IDF (метрика, которая показывает насколько важно/релевантно слово в заданном документе). Считается для каждого текста относительно группы текстов по топику. Для каждого документа записываются 5 самых релевантных слов (key-words) в DataFrame.

In [83]:
for i in tqdm(range(21)):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([' '.join(text) for text in list(df.loc[df['topic'] == i, 'lemmas'])])
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    denselist = dense.tolist()
    df_tfidf = pd.DataFrame(denselist, columns=feature_names).T
    df_tfidf.columns = list(df.index[df['topic'] == i])
    cols = df_tfidf.columns
    for f in cols:
        best = df_tfidf[f].sort_values(ascending=False).head().index.tolist()
        df.loc[f, 'tf_idf_words'] = [[k] for k in best]

100%|██████████| 21/21 [04:54<00:00, 14.03s/it]


In [85]:
df.to_csv('result_mails.csv')

Программа создаёт из DataFrame файл .csv, но я не могу прикрепить его на GitHub, потому что он слишком тяжёлый. Поэтому покажу здесь.

In [86]:
df

,content,topic,lemmas,tf_idf_words
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,11.0,"[where, s, thing, car, nntp_posting, host, lin...","[car, door, lerxst, funky, front_bumper]"
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,9.0,"[guy_kuo, summary, final, call, report, accele...","[upgrade, clock, guy_kuo, add, experience]"
10,From: irwin@cmptrc.lonestar.org (Irwin Arnstei...,6.0,"[defence, technology, organisation, line, writ...","[fractally, recognisable, tv_station, organisa..."
100,From: tchen@magnus.acs.ohio-state.edu (Tsung-K...,16.0,"[earl_wallace, friend, platform, software, lin...","[gun, people, bulky, allowed, earl_wallace]"
1000,From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)\n...,6.0,"[organization, line, host, write, show, exampl...","[domestication, behavior, wild, domesticate, a..."
10000,From: a207706@moe.dseg.ti.com (Robert Loper)\n...,6.0,"[write, help, wife, inform, wants_convertible,...","[convertible, car, wife, range, ds]"
10001,From: kimman@magnus.acs.ohio-state.edu (Kim Ri...,6.0,"[steven_walz, many, homosexual, organization, ...","[law, charge, beat, deter, straight]"
10002,From: kwilson@casbah.acns.nwu.edu (Kirtley Wil...,6.0,"[survivor, reply, distribution, bench, line, w...","[blast, fatigue, people, knock, hell]"
10003,Subject: Re: Don't more innocents die without ...,6.0,"[loral, line, write, hate, bike, start, week, ...","[plastic, compound, rubbing, work, loral]"
10004,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,11.0,"[game, game, game, line, electronic, art, disk...","[game, best_offer, art, electronic, original]"


## Coherence score

**Coherence score** измеряет оценку одного топика с помощью измерения степени семантического сходства между словами, которые были оценены высоко в этом топике (то есть считаются "релевантными"). 
**Coherence score типа c_v** работает на основе метода скользящего окна ("алгоритм трансформации, позволяющий сформировать из членов временного ряда набор данных, который может служить обучающим множеством для построения модели прогнозирования" (https://wiki.loginom.ru/articles/windowing-method.html)), сегментации наиболее релевантных слов и мере, которая использует нормализованную точечную взаимную информацию (NPMI)(https://en.wikipedia.org/wiki/Pointwise_mutual_information) и косинусное сходство.
Источник: https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0.